<a href="https://colab.research.google.com/github/IgorCzudy/whatsapp_message_clasterization/blob/main/notebooks/GC_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
! pip install transformers datasets evaluate
! pip install accelerate -U

# Prepare data

In [33]:
import pandas as pd
music_aug = pd.read_csv("https://raw.githubusercontent.com/IgorCzudy/whatsapp_message_clasterization/main/finalData/music_augmented.csv")
party_aug = pd.read_csv("https://raw.githubusercontent.com/IgorCzudy/whatsapp_message_clasterization/main/finalData/party_augmented.csv")
full = pd.read_csv("https://raw.githubusercontent.com/IgorCzudy/whatsapp_message_clasterization/main/CleanChats/FullChat.csv")

In [34]:
full.rename(columns={"Message": "message", "Section": "section"}, inplace=True)

In [35]:
full = full.drop(columns=["Date", "Time", "Sender"])

sections = ["Football", "Music", "Party","General"]
final_df = full[full["section"].isin(sections)]

music_aug["section"] = "Music"
party_aug["section"] = "Party"

final_df = pd.concat([final_df, music_aug, party_aug])

In [36]:
# remove duplicates
final_df = final_df.drop_duplicates()
final_df = final_df.reset_index(drop=True)

In [37]:
final_df["section"].value_counts()

Football    3415
Party       2715
General     2636
Music       1480
Name: section, dtype: int64

In [38]:
final_df=final_df.rename(columns={'section': 'label','message':'text'})
final_df['label'] = final_df['label'].replace('General', 0)
final_df['label'] = final_df['label'].replace('Football', 1)
final_df['label'] = final_df['label'].replace('Party', 2)
final_df['label'] = final_df['label'].replace('Music', 3)
final_df.head()

,text,label
0,Someone going to the beach this afternoon?,0
1,We'll be a few minutes late \nSiuuuuuuu,0
2,locatie: https://maps.google.com/?q=39.473291...,0
3,"Hii, I also will be a few minutes late for th...",0
4,Me too,0


In [39]:

final_df.describe()

,label
count,10246.000000
mean,1.296604
std,1.006064
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,3.000000


## Tokenize

In [40]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from datasets import Dataset

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

data = Dataset.from_pandas(final_df)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_dataset = data.map(preprocess_function, batched=True)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/10246 [00:00<?, ? examples/s]

In [41]:
tokenized_dataset[1]


{'text': " We'll be a few minutes late \nSiuuuuuuu",
 'label': 0,
 'input_ids': [101,
  2057,
  1005,
  2222,
  2022,
  1037,
  2261,
  2781,
  2397,
  9033,
  2226,
  2226,
  2226,
  2226,
  2226,
  2226,
  2226,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [42]:
splits = tokenized_dataset.train_test_split(test_size=0.3, shuffle=True)
dataset_train = splits["train"]
dataset_test = splits["test"]

# Model

In [43]:
# track accuracy during training
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Login to publish model to hugginface


In [44]:
from huggingface_hub import notebook_login

notebook_login()

In [46]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

id2label = {0: "GENERAL", 1: "FOOTBALL", 2: "PARTY", 3: "MUSIC"}
label2id = {"GENERAL": 0, "FOOTBALL": 1, "PARTY": 2, "MUSIC": 3}

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="whatsapp-group-classifierv2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.457075,0.835719
2,0.572500,0.438389,0.831165
3,0.317000,0.555731,0.842876
4,0.204200,0.611275,0.841900


TrainOutput(global_step=1796, training_loss=0.33210816627621387, metrics={'train_runtime': 436.5236, 'train_samples_per_second': 65.719, 'train_steps_per_second': 4.114, 'total_flos': 859924037114688.0, 'train_loss': 0.33210816627621387, 'epoch': 4.0})

To publish to hugginface need login with the access token

In [50]:
#trainer.push_to_hub()

events.out.tfevents.1700663078.f7bbc272f3e8.205.1:   0%|          | 0.00/6.54k [00:00<?, ?B/s]

'https://huggingface.co/DTempo/whatsapp-group-classifierv2/tree/main/'

## Inference

In [52]:
from transformers import pipeline
import transformers
#Change model to the one you want to use
model = "Dtempo/whatsapp-group-classifierv2"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline("text-classification",model=model)

sequences = pipeline(["Hey everyone! There's a jazz concert at Jimmy Glass Jazz Bar this Friday night. Who's up for some  tunes?",
                      "Lest go do some running tomorrow ",
                      "Hello, any plans for tonight? I heard Bamboo Pub has a free entry until 00:30. What do you think?"])
for seq in sequences:
    print(f"Result: {seq}")


Result: {'label': 'MUSIC', 'score': 0.9938617944717407}
Result: {'label': 'FOOTBALL', 'score': 0.8408170342445374}
Result: {'label': 'PARTY', 'score': 0.9895837903022766}
